# Tutorial 03: weak imposition of Dirichlet BCs by a Lagrange multiplier (linear problem)

In this tutorial we solve the problem

$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = g, & \text{on } \Gamma = \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D.

We compare the following two cases:
* **strong imposition of Dirichlet BCs**:
the corresponding weak formulation is
$$
\text{find } u \in V_g \text{ s.t. } \int_\Omega \nabla u \cdot \nabla v = \int_\Omega f v, \quad \forall v \in V_0\\
$$
where
$$
V_g = \{v \in H^1(\Omega): v|_\Gamma = g\},\\
V_0 = \{v \in H^1(\Omega): v|_\Gamma = 0\}.\\
$$
* **weak imposition of Dirichlet BCs**: this requires an introduction of a multiplier $\lambda$ which is restricted to $\Gamma$, and solves
$$
\text{find } w, \lambda \in V \times M \text{ s.t. }\\
\begin{cases}
\int_\Omega \nabla w \cdot \nabla v + \int_\Gamma \lambda v = \int_\Omega f v, & \forall v \in V,\\
\int_\Gamma w \mu = \int_\Gamma g \mu, & \forall \mu \in M
\end{cases}
$$
where
$$
V = H^1(\Omega),\\
M = L^{2}(\Gamma).\\
$$

This example is a prototypical case of problems containing subdomain/boundary restricted variables (the Lagrange multiplier, in this case).

In [ ]:
from numpy import arange, isclose, sin, sqrt, where
from petsc4py import PETSc
from ufl import grad, inner, Measure
from dolfinx import *
from dolfinx.cpp.mesh import GhostMode
from dolfinx.fem import assemble_scalar, locate_dofs_topological
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot
from multiphenics import *

### Mesh

In [ ]:
with XDMFFile(MPI.comm_world, "data/circle.xdmf") as infile:
    mesh = infile.read_mesh(GhostMode.none)
with XDMFFile(MPI.comm_world, "data/circle_subdomains.xdmf") as infile:
    subdomains = infile.read_mf_size_t(mesh)
with XDMFFile(MPI.comm_world, "data/circle_boundaries.xdmf") as infile:
    boundaries = infile.read_mf_size_t(mesh)
facets_Gamma = where(boundaries.values == 1)[0]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

### Weak imposition of Dirichlet BCs

In [ ]:
# Define a block function space
V = FunctionSpace(mesh, ("Lagrange", 2))
dofs_V = arange(0, V.dofmap.index_map.block_size*(V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts))
dofs_V_Gamma = locate_dofs_topological(V, boundaries.dim, facets_Gamma)
W = BlockFunctionSpace([V, V], restrict=[dofs_V, dofs_V_Gamma])

In [ ]:
# Define trial and test functions
ul = BlockTrialFunction(W)
(u, l) = block_split(ul)
vm = BlockTestFunction(W)
(v, m) = block_split(vm)

In [ ]:
# Define problem block forms
g = Function(V)
g.interpolate(lambda x: sin(3*x[0] + 1)*sin(3*x[1] + 1))
a = [[inner(grad(u), grad(v))*dx, l*v*ds],
     [u*m*ds                    , 0     ]]
f =  [v*dx                      , g*m*ds]
A = BlockForm2(a, [W, W])
F = BlockForm1(f, [W])

In [ ]:
# Solve
ul = BlockFunction(W)
solver_parameters = {"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"}
block_solve(A, ul, F, petsc_options=solver_parameters)

In [ ]:
plot(ul[0])

### Strong imposition of Dirichlet BCs

In [ ]:
# Define Dirichlet BC object on Gamma
bc_ex = DirichletBC(g, dofs_V_Gamma)

In [ ]:
# Solve
u_ex = Function(V)
solve(a[0][0] == f[0], u_ex, bc_ex, petsc_options=solver_parameters)
u_ex.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
plot(u_ex)

### Comparison and error compuation

In [ ]:
u_ex_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(grad(u_ex), grad(u_ex))*dx)))
err_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(grad(u_ex - ul[0]), grad(u_ex - ul[0]))*dx)))
print("Relative error is equal to", err_norm/u_ex_norm)
assert isclose(err_norm/u_ex_norm, 0., atol=1.e-10)